In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio

In [ ]:
import os
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

import seaborn as sns
import cv2
import glob

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.11.0


In [ ]:
import warnings
import sys
from tqdm import tqdm

import random
from sklearn.model_selection import train_test_split
%matplotlib inline

from urllib.parse import urlparse
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, BatchNormalization, Dropout, Activation, Concatenate

from tensorflow.keras import backend as K, Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow.keras as keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, Callback , ModelCheckpoint
from tensorflow.keras.metrics import Accuracy,binary_crossentropy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
csv_path = "/content/drive/MyDrive/Genetic_Algorithm_CNN_Project/Folds.csv"
dir= "/content/drive/MyDrive/Genetic_Algorithm_CNN_Project/BreaKHis_v1/"

In [ ]:
def dataset(csv_path, classes, dir_path):
  df = pd.read_csv(csv_path)

  df["label"] = df['filename'].apply(lambda x: x.split("/")[3])
  df['class'] = df['label'].apply(lambda x: 0 if x =='benign' else 1)

  train_df = df[df['grp'] == 'train']
  test_df = df[df['grp'] == 'test']

  train_df = train_df.sample(frac = 1).reset_index()
  test_df = test_df.sample(frac = 1).reset_index()

  train_df['filename'] = train_df['filename'].apply(lambda x: os.path.join(dir_path, x))
  test_df['filename'] = train_df['filename'].apply(lambda x: os.path.join(dir_path, x))

  return train_df, test_df

In [ ]:
classes = ["benign","malign"]
train_df, test_df = dataset(csv_path, classes, dir)

In [ ]:
train_df = train_df.iloc[:150, :]

In [ ]:
test_df = test_df.iloc[:30, :]

## Data Generator

In [ ]:
class CustomImageGenerator(keras.utils.Sequence):
    def __init__(self, image_paths, targets, batch_size, image_size):
        self.image_paths = image_paths
        self.targets = targets
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = len(np.unique(targets))
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.image_paths) / float(self.batch_size)))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_image_paths = [self.image_paths[i] for i in batch_indexes]
        batch_targets = [self.targets[i] for i in batch_indexes]
        batch_images = np.zeros((len(batch_image_paths), *self.image_size, 3))
        for i, path in enumerate(batch_image_paths):
            img = keras.preprocessing.image.load_img(path, target_size=self.image_size)
            img = keras.preprocessing.image.img_to_array(img)
            img /= 255.0
            batch_images[i] = img
        batch_targets =  keras.utils.to_categorical(batch_targets, num_classes=self.num_classes)
        return batch_images, batch_targets[:,0]
    
    def on_epoch_end(self):
        np.random.shuffle(self.indexes)


In [ ]:
traingen = CustomImageGenerator(train_df['filename'].values, train_df['class'].values, batch_size=32, image_size= (256,256))
valgen = CustomImageGenerator(test_df['filename'].values, test_df['class'].values, batch_size=32, image_size= (256,256))

In [ ]:
X, y = next(iter(traingen))

In [ ]:
X.shape

(32, 256, 256, 3)

In [ ]:
y.shape

(32,)

In [ ]:
y

array([0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
X_val, y_val = next(iter(valgen))

In [ ]:
X_val.shape

(32, 256, 256, 3)

In [ ]:
y_val.shape

(32,)

In [ ]:
y_val

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
      dtype=float32)

# Simple CNN Network

In [ ]:
# input_layer = tf.keras.Input(shape=(256,256, 3))

# x = Conv2D(64, (2,2), activation='relu')(input_layer)
# x = MaxPooling2D((2,2))(x)
# x = Conv2D(32, (2,2), activation='relu')(x)
# x = MaxPooling2D((2,2))(x)
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.4)(x)

# output_layer = Dense(1, activation='sigmoid')(x)

# fuc_model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# fuc_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 255, 255, 64)      832       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 64)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      8224      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 127008)            0   

In [ ]:
# fuc_model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# history = fuc_model.fit(traingen,
#                   batch_size = 32, 
#                   epochs=30,
#                   validation_data = valgen, verbose=1)

Epoch 1/30
7/7 [==============================] - 75s 11s/step - loss: 1.1543 - accuracy: 0.5500 - val_loss: 0.7023 - val_accuracy: 0.5800
Epoch 2/30
7/7 [==============================] - 4s 615ms/step - loss: 0.6043 - accuracy: 0.6850 - val_loss: 0.6721 - val_accuracy: 0.5800
Epoch 3/30
7/7 [==============================] - 5s 751ms/step - loss: 0.5722 - accuracy: 0.7300 - val_loss: 0.9277 - val_accuracy: 0.6000
Epoch 4/30
7/7 [==============================] - 4s 653ms/step - loss: 0.7191 - accuracy: 0.6950 - val_loss: 1.1412 - val_accuracy: 0.5800
Epoch 5/30
7/7 [==============================] - 5s 768ms/step - loss: 0.5771 - accuracy: 0.7350 - val_loss: 0.7017 - val_accuracy: 0.5600
Epoch 6/30
7/7 [==============================] - 4s 708ms/step - loss: 0.6084 - accuracy: 0.7850 - val_loss: 0.6651 - val_accuracy: 0.5800
Epoch 7/30
7/7 [==============================] - 4s 605ms/step - loss: 0.5296 - accuracy: 0.8050 - val_loss: 0.8937 - val_accuracy: 0.6200
Epoch 8/30
7/7 [=====

KeyboardInterrupt: ignored

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 21 not upgraded.
Need to get 420 MB of archives.
After this operation, 1,383 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.4.1.50-1+cuda11.6 [420 MB]
Fetched 420 MB in 6s (71.2 MB/s)
(Reading database ... 128285 files and directories currently installed.)
Removing libcudnn8-dev (8.7.0.84-1+cuda11.8) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.7.0.84-1+cuda11.8 to 8.4.1.50-1+cuda11.6
(Reading database ... 128252 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.4.1.5

In [ ]:
import random
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD

# Correct Code of Genetic Algorithm

In [ ]:
import random

# Define the parameter values for the CNN model
parameter_values = {
    'dropout': [0, 0.25, 0.5],
    'optimizer': ['Adam', 'RMSprop', 'SGD'],
    'batch_size': [16, 32, 64],
    'epochs': [2, 3, 5],
    'activation': ['relu', 'sigmoid', 'tanh']
}

# Define the size of the population and the number of generations
POPULATION_SIZE = 10
NUM_GENERATIONS = 5

# Define the fitness function for evaluating each individual in the population
def evaluate_fitness(individual):
    # Define the CNN model using the parameters in the individual
    fitness_scores = []
    model = keras.Sequential([
        keras.layers.Conv2D(filters=128, kernel_size=3, activation=individual['activation'], padding = 'same', input_shape=(256, 256, 3)),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=3, padding = 'same', activation=individual['activation']),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(individual['dropout']),
        keras.layers.Conv2D(filters=32, kernel_size=3, padding = 'same', activation=individual['activation']),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(individual['dropout']),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation=individual['activation']),
        keras.layers.Dense(64, activation=individual['activation']),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=individual['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])
    
    # epochs_str = individual['epochs']
    # epochs_int = int(epochs_str)
    # individual['epochs'] = epochs_int
    # Define the early stopping callback
    # early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, mode='max', verbose=1)

    history = model.fit(traingen, batch_size=individual['batch_size'], epochs=individual['epochs'], validation_data=(valgen))
    fitness_scores.append(history.history['val_accuracy'][-1])
    # fitness_score = random.uniform(0, 1)  # This is just a placeholder for demonstration purposes
    return fitness_scores

# Define the selection function for selecting parents from the population
def select_parents(population):
    # Here you could use any selection method you like, such as tournament selection, roulette wheel selection, etc.
    # For simplicity, let's just select two random individuals from the population
    parent1, parent2 = random.sample(population, 2)
    return [parent1, parent2]

# Define the crossover function for combining the genes of two parents to create a child
def crossover(parent1, parent2):
    # Here you could use any crossover method you like, such as single-point crossover, uniform crossover, etc.
    # For simplicity, let's just randomly select genes from the two parents to create a new individual
    child = {}
    for param in parameter_values:
        if isinstance(parameter_values[param], list):
            child[param] = random.choice([parent1[param], parent2[param]])
        else:
            child[param] = parent1[param] if random.random() < 0.5 else parent2[param]
    return child

# Define the mutation function for introducing random changes in an individual's genes
def mutate(params):
    # Here you could use any mutation method you like, such as random resetting, Gaussian mutation, etc.
    # For simplicity, let's just randomly choose a gene and replace its value with a random value from the parameter values
    # param_to_mutate = random.choice(list(parameter_values.keys()))
    individual = {}
    # if isinstance(parameter_values[params], list):
    #   individual[params] = random.choice(parameter_values[params])

    if isinstance(parameter_values[params], list):
        individual = {params: random.choice(parameter_values[params])}
    else:
        individual = {params: parameter_values[params][random.randint(0, len(parameter_values[params])-1)]}
    
    return individual[params]

# # Generate an initial population of individuals with random parameter values
# population = []
# for i in range(POPULATION_SIZE):
#     individual = {}
#     for param in parameter_values:
#         if isinstance(parameter_values[param], list):
#             individual[param] = random.choice(parameter_values[param])
#         else:
#             individual[param] = parameter_values[param][random.randint(0, len(parameter_values[param])-1)]
#     population.append(individual)

# # Evolve the population for the specified number of generations
# for generation in range(NUM_GENERATIONS):
#     print(f"Generation {generation + 1}")
#     # Evaluate the fitness of each individual in the population
#     fitness_scores = [evaluate_fitness(individual) for individual in population]

    # Select parents and create a new population through


In [ ]:
population = []
POPULATION_SIZE = 10
for i in range(POPULATION_SIZE):
    individual = {}
    for param in parameter_values:
        if isinstance(parameter_values[param], list):
            individual[param] = random.choice(parameter_values[param])
        else:
            individual[param] = parameter_values[param][random.randint(0, len(parameter_values[param])-1)]
    population.append(individual)
child = {}
offspring = []
for param in parameter_values:
  print(param)
  if random.random() < 0.1:
    child[param] = mutate(param)
    # print(child[param])
  # else:
    # child[param] = random.choice([parent1[param], parent2[param]])
  print(f"Child: {child}")
  offspring.append(child)

dropout
Child: {}
optimizer
Child: {}
batch_size
Child: {}
epochs
Child: {}
activation
Child: {}


In [ ]:
random.random()

0.04743757764301415

In [ ]:
population = []
POPULATION_SIZE = 10
for i in range(POPULATION_SIZE):
    individual = {}
    for param in parameter_values:
        if isinstance(parameter_values[param], list):
            individual[param] = random.choice(parameter_values[param])
        else:
            individual[param] = parameter_values[param][random.randint(0, len(parameter_values[param])-1)]
    population.append(individual)
NUM_GENERATIONS = 5
for generation in range(NUM_GENERATIONS):
    print(f"Generation {generation + 1}")
    # Evaluate the fitness of each individual in the population
    # fitness_scores = [evaluate_fitness(individual) for individual in population]

    parents = select_parents(population)
    # print(parents)
    # print(len(parents))
    # fitness_scores = [0.3, 0.4, 0.7, 0.1, 0.6, 0.8, 0.5, 0.3]
    offspring = []
    for i in range(6):
      parent1, parent2 = random.sample(parents, 2)
      # print(f"Parent1: {parent1}")
      # print(f"Parent2: {parent2}")
      child = {}
      for param in parameter_values:
        if random.random() < 0.1:
          child[param] = mutate(param)
        else:
          child[param] = random.choice([parent1[param], parent2[param]])
        # print(f"Child: {child}")
        offspring.append(child)
      
      # print(f'Offspring: {offspring}')
      # print(len(offspring))
      new_population = parents + offspring
      # if new_population['epochs'] == {}:
      #   # new_population['epochs'] = new_population['epochs']
      #   print(new_population['epochs']['epochs'])
      print(f'new_population: {new_population}')
      print(len(new_population))
      old_population = population
      # population = select_population(new_population, fitness_scores, 10) 
      # best_fitness = max(fitness_scores, key=lambda x: x[1])[1]

      # print(f"Generation {g}: Best validation accuracy = {best_fitness}")
                
      # # Save the best hyperparameters and their corresponding validation accuracy to the file
      # best_parameters = max(fitness_scores, key=lambda x: x[1])[0]
      # writer.writerow({'Generation': g, 
      #               'Epochs': best_parameters['epochs'], 
      #               'Batch Size': best_parameters['batch_size'], 
      #               'Optimizer': best_parameters['optimizer'], 
      #               'Accuracy': best_fitness
      #           })
      # if best_fitness > 0.97:
      #   break

Generation 1
new_population: [{'dropout': 0, 'optimizer': 'SGD', 'batch_size': 16, 'epochs': 3, 'activation': 'sigmoid'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'relu'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'sigmoid'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'sigmoid'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'sigmoid'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'sigmoid'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'sigmoid'}]
7
new_population: [{'dropout': 0, 'optimizer': 'SGD', 'batch_size': 16, 'epochs': 3, 'activation': 'sigmoid'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'relu'}, {'dropout': 0, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 2, 'activation': 'sigmoid'}, {'dro

In [ ]:
import csv

# Define the main genetic algorithm function
def genetic_algorithm(parameter_values, population_size, num_parents, mutation_rate, generations, save_file):
    # Initialize the population with random hyperparameters
    # population = []
    population = []
    for i in range(population_size):
      individual = {}
      for param in parameter_values:
        if isinstance(parameter_values[param], list):
          individual[param] = random.choice(parameter_values[param])
        else:
          individual[param] = parameter_values[param][random.randint(0, len(parameter_values[param])-1)]
        population.append(individual)
    # NUM_GENERATIONS = 5
    num_offspring = population_size - num_parents
    # Open a CSV file to write the results
    with open(save_file, mode='w') as result_file:
        fieldnames = ['Generation', 'Epochs', 'Batch Size', 'Optimizer', 'Accuracy']
        writer = csv.DictWriter(result_file, fieldnames=fieldnames)
        writer.writeheader()

        for generation in range(generations):
            print(f"Generation {generation + 1}")
            # Evaluate the fitness of each individual in the population
            # [(parameters, fitness_function(parameters)) for parameters in population]
            fitness_scores = [(individual, evaluate_fitness(individual)) for individual in population]

            parents = select_parents(population)
            # print(parents)
            # print(len(parents))
            # fitness_scores = [0.3, 0.4, 0.7, 0.1, 0.6, 0.8, 0.5, 0.3]
            offspring = []
            for i in range(num_offspring):
              parent1, parent2 = random.sample(parents, 2)
              # print(f"Parent1: {parent1}")
              # print(f"Parent2: {parent2}")
              child = {}
              for param in parameter_values:
                if random.random() < mutation_rate:
                  child[param] = mutate(param)
                else:
                  child[param] = random.choice([parent1[param], parent2[param]])
                # print(f"Child: {child}")
                offspring.append(child)
              
              # print(f'Offspring: {offspring}')
              # print(len(offspring))
              population = parents + offspring
              # print(f'new_population: {population}')
              # print(len(new_population))
              # old_population = population
              # population = select_population(new_population, fitness_scores, 10) 
            best_fitness = max(fitness_scores, key=lambda x: x[1])[1]

            print(f"Generation {generation}: Best validation accuracy = {best_fitness}")
                        
              # Save the best hyperparameters and their corresponding validation accuracy to the file
            best_parameters = max(fitness_scores, key=lambda x: x[1])[0]
            writer.writerow({'Generation': generation, 
                            'Epochs': best_parameters['epochs'], 
                            'Batch Size': best_parameters['batch_size'], 
                            'Optimizer': best_parameters['optimizer'], 
                            'Accuracy': best_fitness
                        })
              # if best_fitness > 0.97:
              #   break
    # Return the best individual from the final population
    best_parameters = max(fitness_scores, key=lambda x: x[1])[0]
    return best_parameters


In [ ]:
best_params = genetic_algorithm(parameter_values, population_size=10, num_parents=4, mutation_rate=0.1, generations=3, save_file ='result.csv')

Generation 1
Epoch 1/5
5/5 [==============================] - 72s 11s/step - loss: 0.8583 - accuracy: 0.3533 - val_loss: 0.7515 - val_accuracy: 0.2333
Epoch 2/5
5/5 [==============================] - 4s 666ms/step - loss: 0.7024 - accuracy: 0.4267 - val_loss: 0.6639 - val_accuracy: 0.7667
Epoch 3/5
5/5 [==============================] - 4s 880ms/step - loss: 0.6681 - accuracy: 0.6467 - val_loss: 0.6235 - val_accuracy: 0.7667
Epoch 4/5
5/5 [==============================] - 4s 666ms/step - loss: 0.6552 - accuracy: 0.6467 - val_loss: 0.6078 - val_accuracy: 0.7667
Epoch 5/5
5/5 [==============================] - 3s 723ms/step - loss: 0.6538 - accuracy: 0.6467 - val_loss: 0.5985 - val_accuracy: 0.7667
Epoch 1/5
5/5 [==============================] - 6s 1s/step - loss: 0.7712 - accuracy: 0.3600 - val_loss: 0.6810 - val_accuracy: 0.7667
Epoch 2/5
5/5 [==============================] - 3s 633ms/step - loss: 0.6687 - accuracy: 0.6467 - val_loss: 0.6237 - val_accuracy: 0.7667
Epoch 3/5
5/5 [===

5/5 [==============================] - 6s 1s/step - loss: 0.7331 - accuracy: 0.5267 - val_loss: 0.5948 - val_accuracy: 0.7667
Epoch 2/3
5/5 [==============================] - 4s 717ms/step - loss: 0.6623 - accuracy: 0.6467 - val_loss: 0.5666 - val_accuracy: 0.7667
Epoch 3/3
5/5 [==============================] - 4s 675ms/step - loss: 0.6498 - accuracy: 0.6467 - val_loss: 0.5859 - val_accuracy: 0.7667
Epoch 1/3
5/5 [==============================] - ETA: 0s - loss: 0.7338 - accuracy: 0.6467

5/5 [==============================] - 7s 861ms/step - loss: 0.7338 - accuracy: 0.6467 - val_loss: 0.5506 - val_accuracy: 0.7667
Epoch 2/3
5/5 [==============================] - 3s 646ms/step - loss: 0.6575 - accuracy: 0.6467 - val_loss: 0.5593 - val_accuracy: 0.7667
Epoch 3/3
5/5 [==============================] - 4s 664ms/step - loss: 0.6510 - accuracy: 0.6467 - val_loss: 0.5904 - val_accuracy: 0.7667
Epoch 1/5
5/5 [==============================] - 6s 782ms/step - loss: 0.7612 - accuracy: 0.5667 - val_loss: 0.5605 - val_accuracy: 0.7667
Epoch 2/5
5/5 [==============================] - 3s 633ms/step - loss: 0.6585 - accuracy: 0.6467 - val_loss: 0.5641 - val_accuracy: 0.7667
Epoch 3/5
5/5 [==============================] - 4s 728ms/step - loss: 0.6516 - accuracy: 0.6467 - val_loss: 0.5729 - val_accuracy: 0.7667
Epoch 4/5
5/5 [==============================] - 3s 626ms/step - loss: 0.6617 - accuracy: 0.6467 - val_loss: 0.5677 - val_accuracy: 0.7667
Epoch 5/5
5/5 [======================

In [ ]:
len(population)

10

In [ ]:
parameter_values
for params in parameter_values:
  print(params)

dropout
optimizer
batch_size
epochs
activation


In [ ]:
# child['dropout'] = mutate(parameter_values)

TypeError: ignored

In [ ]:
# # Define the early stopping callback
# early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, mode='max', verbose=1)

# # Train the model with the early stopping callback
# model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stop_callback])

In [ ]:
# parameter_values['batch_size']

[16, 32, 64]

In [ ]:
# param_to_mutate = random.choice(list(parameter_values.keys()))

In [ ]:
# param_to_mutate

'activation'

In [ ]:
# individual[param_to_mutate] = random.choice(parameter_values[param_to_mutate])

In [ ]:
# # individual[param_to_mutate]

# if isinstance(parameter_values[param_to_mutate], list):
#   individual[param_to_mutate] = random.choice(parameter_values[param_to_mutate])
# else:
#   individual[param_to_mutate] = parameter_values[param_to_mutate][random.randint(0, len(parameter_values[param_to_mutate])-1)]

# individual[param_to_mutate]

'relu'

In [ ]:
# def mutate(params):
#     # Here you could use any mutation method you like, such as random resetting, Gaussian mutation, etc.
#     # For simplicity, let's just randomly choose a gene and replace its value with a random value from the parameter values
#     # param_to_mutate = random.choice(list(parameter_values.keys()))
#     if isinstance(parameter_values[params], list):
#       individual[params] = random.choice(parameter_values[params])
#     else:
#       individual[params] = parameter_values[params][random.randint(0, len(parameter_values[params])-1)]
    
#     return individual

In [ ]:
# child = mutate('dropout')

In [ ]:
# for param in parameter_values:
#     if random.random() < 0.1:
#       child[param] = mutate(param)
#     else:
#       child[param] = random.choice([parent1[param], parent2[param]])
#     print(f"Child: {child}")

Child: {'dropout': 0.5, 'optimizer': 'SGD', 'batch_size': 16, 'epochs': 3, 'activation': 'relu'}
Child: {'dropout': 0.5, 'optimizer': 'Adam', 'batch_size': 16, 'epochs': 3, 'activation': 'relu'}
Child: {'dropout': 0.5, 'optimizer': 'Adam', 'batch_size': 64, 'epochs': 3, 'activation': 'relu'}
Child: {'dropout': 0.5, 'optimizer': 'Adam', 'batch_size': 64, 'epochs': 3, 'activation': 'relu'}
Child: {'dropout': 0.5, 'optimizer': 'Adam', 'batch_size': 64, 'epochs': 3, 'activation': 'relu'}


In [ ]:
# # Select parents
#         parents = select_parents(population, fitness_scores, num_parents)

#         # Generate offspring
#         offspring = []
#         num_offspring = population_size - num_parents
#         for i in range(num_offspring):
#             parent1, parent2 = random.sample(parents, 2)
#             child = {}
#             for param in parameter_values:
#                 if random.random() < mutation_rate:
#                     child[param] = mutate(parameter_values[param])
#                 else:
#                     child[param] = random.choice([parent1[param], parent2[param]])
#             offspring.append(child)

#         # Combine parents and offspring
#         new_population = parents + offspring

#         # Evaluate fitness of new population
#         new_fitness_scores = evaluate_population(new_population, traingen, valgen)

IndentationError: ignored